In [ ]:
import requests
import dotenv 
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

dotenv.load_dotenv()

In [ ]:
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

GEOCODE_ENDPOINT = 'http://api.openweathermap.org/geo/1.0/direct'
AIRPOLLUTION_ENDPOINT = 'http://api.openweathermap.org/data/2.5/air_pollution/history'

startDate = '2024-01-01'
startTimeStamp = dt.datetime.strptime(startDate, '%Y-%m-%d').timestamp()
startTimeStamp = int(startTimeStamp)

endTimeStamp = dt.datetime.now().timestamp()
endTimeStamp = int(endTimeStamp)

def getLatLong(city):
    url = f"{GEOCODE_ENDPOINT}?q={city}&limit=1&appid={OPENWEATHER_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data[0]['lat'], data[0]['lon']
    else:
        print("Error: ", response.text)
        return None, None
    
def getAirPollution(lat, lon):
    url = f"{AIRPOLLUTION_ENDPOINT}?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&start={startTimeStamp}&end={endTimeStamp}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error: ", response.text)
        return None
    

lat, lon = getLatLong("Melbourne")
data = getAirPollution(lat, lon)
print(data)

In [ ]:
lat, lon = getLatLong("Melbourne")
data = getAirPollution(lat, lon)


In [ ]:
dates = [dt.datetime.fromtimestamp(item['dt']) for item in data['list']]
components = [item['components'] for item in data['list']]

df = pd.DataFrame(components, index=dates)
df.index = pd.to_datetime(df.index, unit='D', origin='unix')

In [ ]:
df_daily_avg = df.resample('D').mean()

In [ ]:
df_daily_avg_july = df_daily_avg.loc['2024-06-01':'2024-07-31']

In [ ]:
pollution_metrics = df.columns

def plot_metric_time_series (df, metric):
    plt.figure(figsize=(10, 5))
    plt.style.use('fivethirtyeight')
    plt.plot(df.index, df[metric], label=metric)
    plt.title(f'Time Series of {metric}')
    plt.xlabel('Date')
    plt.xticks(rotation=60)
    plt.ylabel(f'Concentration of {metric} (μg/m3)')
    plt.legend()
    plt.show()

plot_metric_time_series(df, pollution_metrics[0])
plot_metric_time_series(df_daily_avg, pollution_metrics[0])
plot_metric_time_series(df_daily_avg_july, metric=pollution_metrics[0])